In [1]:
#importing required libraries

import random
import json
import pickle
import numpy as np

import nltk 
from  nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


In [2]:
#initializing lemmatizer= for tokenizing words
lemmatizer=WordNetLemmatizer()

#loading the intents file
intents=json.loads(open('intents.json').read())

In [3]:
#the following lists are for storing:
# words -> tokenize(lemmatized)
# classes -> tags
# documents -> words+tags

words=[]
classes=[]
documents=[]

#cleaning words
ignore_letters=['?', "!", ".",","]

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# intents -> pattern -> tokenize -> doc (token+tag) -> class=tag

for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list=nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [6]:
 nltk.download('wordnet')
 nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [7]:
#remove ignored letters
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

#removing duplicates
words=sorted(set(words))

print(words)

['App', 'Are', 'Artificial', 'Can', 'Client', 'Development', 'Do', 'Good', 'Greetings', 'Hello', 'Help', 'Hi', 'I', 'Intelligence', 'Is', 'May', 'Web', 'What', 'Where', 'Who', 'Working', 'a', 'about', 'afternoon', 'am', 'are', 'area', 'assistance', 'available', 'brief', 'business', 'client', 'company', 'created', 'developer', 'development', 'discussion', 'do', 'doe', 'expertise', 'give', 'have', 'history', 'hour', 'in', 'information', 'inquire', 'interested', 'international', 'is', 'it', 'know', 'like', 'list', 'located', 'location', 'me', 'meet', 'more', 'morning', 'name', 'need', 'now', 'of', 'offer', 'office', 'on', 'past', 'person', 'physically', 'possible', 'project', 'provide', 'seeking', 'send', 'service', 'situated', 'speak', 'speaking', 'take', 'tell', 'to', 'type', 'want', 'who', 'with', 'work', 'worked', 'working', 'would', 'you', 'your']


In [8]:
#removing duplicates (from classes)
classes=sorted(set(classes))

#storing the words and classes using pickle
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


In [9]:
#next is converting the words/tokens into numerical rep for neural networks(training)

training = []

#for template of 0s (as of total classes)
output_empty = [0] * len(classes)

#iterating documents to find the combination of words (1 hot rep of 1s or 0s)
for document in documents:

  #to be stored in bag
    bag = []

  #first element has words if you print document to check
    word_patterns = document[0]
   
  #perform lemmatization operation
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

  #iterate words list  
    for word in words:

  #checks if the word is present in both lists then bag=1 else bag=0
        bag.append(1) if word in word_patterns else bag.append(0)
    
  #copying list
    output_row = list(output_empty)
  
  #obtaining tags from document
    output_row[classes.index(document[1])] = 1
   
  # one hot rep of words are paired here 
    training.append([bag, output_row])

#shuffling before training
random.shuffle(training)

#converting list to np array
training = np.array(training)

#dividing the data for training (x for the first element of list and likewise y for second element)
train_x = list(training[:, 0])
train_y = list(training[:, 1])

#building NN
model = Sequential()
model.add(
    Dense(128, input_shape=(len(train_x[0]), ), activation='relu') 
)
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

#using optimizer
sgd = SGD(lr=0.01,decay=1e-6,momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#training with fit function
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

#saving the trained model
model.save('chatbot_model.h5', hist)

print('Done')

<ipython-input-9-417d81a41ac1>:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


8/8 [==============================] - 1s 3ms/step - loss: 2.1497 - accuracy: 0.1892
Epoch 2/200
8/8 [==============================] - 0s 2ms/step - loss: 2.0082 - accuracy: 0.2432
Epoch 3/200
8/8 [==============================] - 0s 3ms/step - loss: 1.9311 - accuracy: 0.3243
Epoch 4/200
8/8 [==============================] - 0s 3ms/step - loss: 1.8075 - accuracy: 0.4324
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 1.8202 - accuracy: 0.4595
Epoch 6/200
8/8 [==============================] - 0s 3ms/step - loss: 1.7319 - accuracy: 0.4324
Epoch 7/200
8/8 [==============================] - 0s 3ms/step - loss: 1.6191 - accuracy: 0.5405
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6206 - accuracy: 0.5135
Epoch 9/200
8/8 [==============================] - 0s 3ms/step - loss: 1.4242 - accuracy: 0.6216
Epoch 10/200
8/8 [==============================] - 0s 2ms/step - loss: 1.3094 - accuracy: 0.6216
Epoch 11/200
8/8 [=======================

# building chatbot application that uses trained file

In [10]:
from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

#loading all the dumped files
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

#loading model
model = load_model('chatbot_model.h5')


clean -> bag find -> class find -> response (output)

In [12]:
# tokenize->lemmatize

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [13]:
# sentence->bag of words (0s or 1s)

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [14]:
#predicting sentence's class: 

def predict_class(sentence):
    bow = bag_of_words(sentence)

  #model result
    res = model.predict(np.array([bow]), verbose = 0)[0]

  #setting uncertainity threshold
    ERROR_THRESHOLD = 0.25

  #filtering results
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  #highest probab first
    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []
    
  #returning intents and probab
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [15]:
#getting response

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [19]:
while True:
    message =input("")
    if message=="quit":
      break
    else: 
      ints = predict_class(message)
      res = get_response(ints, intents)
      print(res)

hello
Good to see you, what information are you looking for?
do you provide artificial intelligence servicess?
We welcome both national and international clients.
Who created you
You are speaking with a bot.
what are your working hours?
Yes, we are available 24/7 all year round.
quit
